In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
df = pd.read_csv("logs/monitor_data.csv")
df.timestamp = pd.to_datetime(df.timestamp)


# find line with start time, extract it and remove it from the dataframe

start_time_index= df.number_name=="start_time"
assert sum(start_time_index)==1

#start_time = datetime.fromtimestamp(
#    df.number[start_time_index].values[0])
start_time = df.timestamp[start_time_index].values[0]

df.drop(df.index[start_time_index], inplace=True)



df.index= (df.timestamp-start_time).dt.total_seconds()
df.drop("timestamp", axis=1, inplace=True)
df.head()

In [ ]:
df.query('number_name=="Processing_time" & category=="Transcription"')[["number","message"]]

In [ ]:
Number_categories = ["delay","Processing_time",'Buffer_size']

f,axe = plt.subplots(len(Number_categories),1,sharex=True,figsize=(1+3*len(Number_categories),5))
assert len(axe)==len(Number_categories)

Axes ={ k:axe[i] for i,k in enumerate(Number_categories)}



for g,d in df.groupby(["category","subcategory","number_name"],dropna=False):
    c,sc,number_name = g
    if number_name not in Number_categories:
        continue
    label = c
    if type(sc)==str:
        label += " " + sc
    print(c,sc,number_name)
    d.number.plot(label=label ,ax=Axes[number_name],marker="o",linestyle="-")

for k in Axes:
    Axes[k].set_title(k)
    Axes[k].set_ylabel("s")
    Axes[k].grid(True)
    Axes[k].legend()


#format x axis as min: sec
axe[1].set_xlabel("Time (s)")
_ = axe[1].set_xticks(np.arange(0,df.index.max(),60))
#axe[1].set_xticklabels([str(x).split(" ")[-1] for x in axe[1].get_xticks()])

In [ ]:
if not df.query('number_name=="Processing_time" and subcategory=="VAC"').number.describe().loc["75%"]< 0.04:

    print("Warning: Processing time is too high")
    print(df.query('number_name=="Processing_time" and subcategory=="VAC"').number.describe())

In [ ]:
df.query("category=='Transcription'")

In [ ]:
d= df.query("category=='Transcription'")

d.index= d.index.round(2)
d.drop(columns=["timestamp","subcategory"],inplace=True)

d=d.pivot_table(values="number",columns=["number_name"],index="timestamp")

In [ ]:
d=df.query("number_name=='Time'")


((d.index-d.number)).plot()

In [ ]:
Number_categories = df.number_name.unique()
N=len(Number_categories)

f,axe = plt.subplots(N,1,sharex=True,figsize=(9,1+3*N))


Axes ={ k:axe[i] for i,k in enumerate(Number_categories)}







for g,d in df.groupby(["category","subcategory","number_name"],dropna=False):
    c,sc,number_name = g
    if number_name not in Number_categories:
        continue
    label = c
    if type(sc)==str:
        label += " " + sc
    # print(c,sc,number_name)
    d.number.plot(label=label ,ax=Axes[number_name],marker="o",linestyle="-")

for k in Axes:
    Axes[k].set_title(k)
    Axes[k].grid(True)
    Axes[k].legend()


#format x axis as min: sec
axe[1].set_xlabel("Time (s)")
_ = axe[1].set_xticks(np.arange(0,df.index.max(),60))
#axe[1].set_xticklabels([str(x).split(" ")[-1] for x in axe[1].get_xticks()])

# Sentences



In [ ]:
S = pd.read_table("translations/sentence_transcript.tsv",header=None)
S.columns = ["Timestamp","Start","End","Sentence"]
S["Duration"]= S.End-S.Start
S["Timestamp"] = pd.to_datetime(S.Timestamp)
S["Pause_after"] = S.Start.shift(-1) - S.End
S["N_chars"] = S.Sentence.str.len()
S["N_words"] = S.Sentence.str.split().str.len()
S.head()

In [ ]:
#S.plot(y="Duration",x="N_chars",kind="scatter")

In [ ]:
S.Pause_after.plot(kind="hist",bins=50)

In [ ]:
for _,row in S.iterrows():
    print(f"{row.Sentence} {row.Pause_after:.2f}s")
    

# Words 

In [ ]:
W= pd.read_csv("translations/transcribed_words.csv",header=None)
W.columns = ["Timestamp","Start","End","Word"]
W["Timestamp"] = pd.to_datetime(W.Timestamp)
W.index = W.Start-W.Start.iloc[0]
W.index.name = "Time"
W["Duration"]= W.End-W.Start
W["Pause_after"] = W.Start.shift(-1) - W.End
W["N_chars"] = W.Word.str.len()
W["N_vowels"] = W.Word.str.count("[aeiouy]")
W["Frequency"] = 60/(W.Duration+ W.Pause_after)


In [ ]:
W.Frequency.rolling(10).mean().plot()
